<a href="https://colab.research.google.com/github/MaryamNourii/Answer-sheet-grader/blob/main/RS_Assistment2017_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install EduData

In [2]:
!edudata download assistment-2017 /content/assistment-2017

downloader, INFO http://base.ustc.edu.cn/data/ASSISTment/anonymized_full_release_competition_dataset.zip is saved as /content/assistment-2017/anonymized_full_release_competition_dataset.zip
downloader, INFO /content/assistment-2017/anonymized_full_release_competition_dataset.zip is unzip to /content/assistment-2017/anonymized_full_release_competition_dataset
/content/assistment-2017/anonymized_full_release_competition_dataset


In [3]:
!unzip -qx /content/assistment-2017/anonymized_full_release_competition_dataset.zip -d ass

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam, schedules, SGD
from keras.metrics import AUC

In [5]:
path = "/content/ass/anonymized_full_release_competition_dataset.csv"
df = pd.read_csv(path, encoding = "ISO-8859-15",low_memory=False)
df

,studentId,MiddleSchoolId,InferredGender,SY ASSISTments Usage,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,...,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING,Ln-1,Ln,MCAS,Enrolled,Selective,isSTEM
0,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.785585,0.000264,0.13,0.061190409,45,0,0,NaN
1,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.000000,0.468252,0.001483,0.061190409,0.213509945,45,0,0,NaN
2,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.000000,0.468252,0.001483,0.116,0.033305768,45,0,0,NaN
3,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.108417,0.010665,0.116,0.033305768,45,0,0,NaN
4,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.000000,0.108417,0.010665,0.033305768,0.118385889,45,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942811,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.000092,0.054517257,0.053783108,28,1,0,NaN
942812,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.633474,0.000000,0.122595,0.002815,0.053783108,0.053673232,28,1,0,NaN
942813,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.101184,0.053783108,0.053673232,28,1,0,NaN
942814,7783,1,NaN,2005-2006,0.210902,0.094888,0.456522,276,0.271812,0.611678,...,0.000000,0.000000,0.122595,0.101184,0.053783108,0.053673232,28,1,0,NaN


In [6]:
data1 = df[['studentId','skill', 'problemId', 'correct']].copy()
data1

,studentId,skill,problemId,correct
0,8,properties-of-geometric-figures,1118,0
1,8,properties-of-geometric-figures,1119,1
2,8,sum-of-interior-angles-more-than-3-sides,1120,0
3,8,sum-of-interior-angles-more-than-3-sides,1120,0
4,8,sum-of-interior-angles-more-than-3-sides,1121,1
...,...,...,...,...
942811,7783,mean,2272,1
942812,7783,mean,2273,0
942813,7783,mean,2273,0
942814,7783,mean,2273,1


In [7]:
data1.groupby('studentId')['problemId'].count().sort_values().groupby(level=0).tail(1)

studentId
5163       2
1868      12
1667      12
6514      19
7296      20
        ... 
5471    2375
954     2496
1079    2593
3609    2742
1483    3057
Name: problemId, Length: 1709, dtype: int64

In [8]:
data = data1[data1['studentId']==1483]
data.reset_index(inplace=True, drop=True)
data

,studentId,skill,problemId,correct
0,1483,proportion,1174,1
1,1483,square-root,1093,0
2,1483,square-root,1094,0
3,1483,square-root,1094,0
4,1483,square-root,1094,0
...,...,...,...,...
3052,1483,discount,1199,0
3053,1483,discount,1199,0
3054,1483,discount,1199,0
3055,1483,discount,1200,1


In [9]:
le = LabelEncoder()
data['skill_index'] = le.fit_transform(data['skill'])
data['skill_index']

<ipython-input-9-9259a7d51b89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['skill_index'] = le.fit_transform(data['skill'])


0       45
1       50
2       50
3       50
4       50
        ..
3052     6
3053     6
3054     6
3055     6
3056     2
Name: skill_index, Length: 3057, dtype: int64

In [10]:
le = LabelEncoder()
data['problem_index'] = le.fit_transform(data['problemId'])
data['problem_index']

<ipython-input-10-6a8d5202e8f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['problem_index'] = le.fit_transform(data['problemId'])


0       629
1       594
2       595
3       595
4       595
       ... 
3052    638
3053    638
3054    638
3055    639
3056    641
Name: problem_index, Length: 3057, dtype: int64

In [11]:
num_skill = len(data['skill'].unique())
num_skill

63

In [12]:
def convert_to_input_vector(qt, at, num_skills):
  input_vector = np.zeros(2*num_skill)
  output_vector = np.zeros(num_skill)

  skill_index = qt
  correct = at
  # if num_skill<100:
  if at:
    answer_index = skill_index
  else:
    answer_index = num_skill + skill_index

  input_vector[answer_index] = 1
  output_vector[skill_index] = at  # else:
  #     input_vector = np.random.normal(size=32)
  return input_vector,output_vector

In [13]:
if num_skill <= 102:
    X = np.zeros((len(data), 2*num_skill))
    Y = np.zeros((len(data), num_skill))
    # for _, group in data.groupby('user_id'):
    current_sequence=[]
    for i, row in data.iterrows():
        input_vector,output_vector = convert_to_input_vector(row['skill_index'], row['correct'], num_skill)
        X[i]= input_vector
        Y[i] = output_vector

In [14]:
X.shape, Y.shape

((3057, 126), (3057, 63))

In [15]:
num_problems=len(data)
num_problems

3057

In [77]:
# from scipy.sparse import csr_matrix
# X = csr_matrix(X)
# X.shape

(942816, 204)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [17]:
X_train=X_train.reshape((X_train.shape[0],1, X_train.shape[1]))
y_train=y_train.reshape((y_train.shape[0],1, y_train.shape[1]))
X_test=X_test.reshape((X_test.shape[0],1, X_test.shape[1]))
y_test=y_test.reshape((y_test.shape[0],1, y_test.shape[1]))
X_val=X_val.reshape((X_val.shape[0],1, X_val.shape[1]))
y_val=y_val.reshape((y_val.shape[0],1, y_val.shape[1]))
y_train.shape,X_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

((2200, 1, 63),
 (2200, 1, 126),
 (612, 1, 126),
 (612, 1, 63),
 (245, 1, 126),
 (245, 1, 63))

In [18]:
inputs = Input(shape=(None, 2*num_skill))
lstm1 = LSTM(200, return_sequences=True, activation="tanh", dropout=0.25)(inputs)
output = Dense(num_skill, activation='sigmoid')(lstm1)
model = Model(inputs=inputs, outputs=output)

In [19]:
epoch=5
# lr = 0.001
# adam = Adam(learning_rate=lr)

# lr = 0.1
# decay_rate = lr / epoch
# momentum = 0.8
# sgd = SGD(learning_rate=lr, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['AUC'])

In [20]:
model.fit(X_train,y_train,batch_size=100, epochs=epoch, validation_data=(X_val, y_val),shuffle=True)

Epoch 1/5
22/22 [==============================] - 10s 54ms/step - loss: 0.6811 - auc: 0.4691 - val_loss: 0.6703 - val_auc: 0.4768
Epoch 2/5
22/22 [==============================] - 2s 97ms/step - loss: 0.6597 - auc: 0.4755 - val_loss: 0.6463 - val_auc: 0.4750
Epoch 3/5
22/22 [==============================] - 0s 6ms/step - loss: 0.6307 - auc: 0.4889 - val_loss: 0.6111 - val_auc: 0.5240
Epoch 4/5
22/22 [==============================] - 0s 6ms/step - loss: 0.5889 - auc: 0.5299 - val_loss: 0.5612 - val_auc: 0.5660
Epoch 5/5
22/22 [==============================] - 0s 7ms/step - loss: 0.5319 - auc: 0.5748 - val_loss: 0.4968 - val_auc: 0.6163


In [21]:
test_loss, test_auc = model.evaluate(X_test, y_test)
print('Test AUC:', test_auc)

8/8 [==============================] - 0s 4ms/step - loss: 0.4969 - auc: 0.5437
Test AUC: 0.5436684489250183


In [22]:
y_pred_test = model.predict(X_test)
y_pred_test[80]

8/8 [==============================] - 2s 169ms/step


array([[0.39711553, 0.4121188 , 0.4168534 , 0.39878672, 0.40259534,
        0.38489002, 0.39374176, 0.4034482 , 0.39665323, 0.39833516,
        0.38787538, 0.4023595 , 0.39988506, 0.3964267 , 0.39474612,
        0.41239575, 0.40164706, 0.39401397, 0.4216203 , 0.3870002 ,
        0.42152604, 0.39826742, 0.40000117, 0.40528303, 0.38378692,
        0.38618332, 0.4212324 , 0.38928345, 0.39986375, 0.3976049 ,
        0.40567026, 0.4061266 , 0.406     , 0.39982194, 0.42087227,
        0.4090816 , 0.39967242, 0.40582335, 0.38768712, 0.40882716,
        0.40757105, 0.4063167 , 0.40316504, 0.39839053, 0.39793482,
        0.39042604, 0.4129055 , 0.40779793, 0.39665172, 0.40651777,
        0.39518648, 0.3935571 , 0.39947382, 0.4024148 , 0.40627506,
        0.40315345, 0.4060458 , 0.40402314, 0.38821924, 0.409633  ,
        0.40404987, 0.4005839 , 0.40033558]], dtype=float32)

In [23]:
model.save('dkt_model.h5')

In [ ]:
# from keras.callbacks import LearningRateScheduler

# lr_init = 1
# lr_decay = 0.0001

# def lr_schedule(epoch):
#     print(lr_init * lr_decay ** epoch)
#     return lr_init * lr_decay ** epoch



# # opt = Adam(learning_rate=lr_init)
# opt = SGD(learning_rate=lr_init)

# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# lr_scheduler = LearningRateScheduler(lr_schedule)

# model.fit(X_train, y_train, batch_size=64, epochs=20, callbacks=[lr_scheduler])

In [ ]:
# from keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=0, min_delta=0.0)

# model.fit(X_train, y_train, batch_size=32, epochs=100,  validation_data=(X_val, y_val),shuffle=True, callbacks=[early_stop])